# Alternating Linear approximation of the ISA (aLISA) method

## Non-linear optimization problem

In [ ]:
import numpy as np
from setup import prepare_argument_dict, prepare_grid_and_dens, print_results

from horton_part import BasisFuncHelper, ExpBasisFuncHelper, LinearISAWPart


### Convex optimization method


In [ ]:
mol, grid, rho = prepare_grid_and_dens("data/h2o.fchk")


def use_cvxopt_solver():
    """Local LISA by solving convex optimization problem."""
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = "cvxopt"
    part = LinearISAWPart(**kwargs)
    part.do_all()
    print_results(part)


use_cvxopt_solver()

### Trust-region method

In [ ]:
def use_trust_region_implicit_constr():
    """Local LISA by solving convex optimization problem."""
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = "trust-region"
    kwargs["solver_options"] = {"explicit_constr": False}
    part = LinearISAWPart(**kwargs)
    part.do_all()
    print_results(part)


use_trust_region_implicit_constr()

## Non-linear equations (fixed-point equations)

### Self-consistent method


In [ ]:
def use_sc_solver():
    """Self-consistent solver."""
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = "sc"
    part = LinearISAWPart(**kwargs)
    part.do_all()
    print_results(part)


use_sc_solver()

### Direct Inversion in Iterative Space (DIIS)

This method has been extensively used to solve self-consistent field (SCF) problems in the fields of quantum chemistry and physics. In this tutorial, we employ this method to accelerate the solving of fixed-point problems.

It should be noted that one potential issue with this method is that non-negative parameters cannot be guaranteed during optimization in the conventional DIIS approach. Although this issue can be addressed by explicitly introducing constraints to the linear combination coefficients, key concepts in DIIS, numerical issues may still arise, such as singular matrices or convergence problems.

In [ ]:
def use_diis_solver():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = "diis"
    part = LinearISAWPart(**kwargs)
    part.do_all()
    print_results(part)


use_diis_solver()

### Newton Method

The final method is the Newton method. Similarly to the previous methods, the Newton method cannot guarantee non-negative parameters, and negative pro-atom densities may arise during optimization. To address this, one might need to modify the hyper-parameters used in the method for different systems, which can make it less robust.

In [ ]:
def newton_method():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = "newton"
    part = LinearISAWPart(**kwargs)
    part.do_all()
    print_results(part)


newton_method()

## Customized Methods

One can also apply customized solvers. Next, we will implement and apply a modified version of the self-consistent method.

In [ ]:
def customize_self_consistent_solver(
    bs_funcs,
    rho,
    propars,
    points,
    weights,
    threshold,
    logger,
    density_cutoff,
    negative_cutoff,
    population_cutoff,
):
    r"""
    Optimize parameters for proatom density functions using a self-consistent (SC) method.

    .. math::

        N_{Ai} = \int \rho_A(r) \frac{\rho_{Ai}^0(r)}{\rho_A^0(r)} dr

    Parameters
    ----------
    bs_funcs : 2D np.ndarray
        Basis functions array with shape (M, N), where 'M' is the number of basis functions
        and 'N' is the number of grid points.
    rho : 1D np.ndarray
        Spherically-averaged atomic density as a function of radial distance, with shape (N,).
    propars : 1D np.ndarray
        Pro-atom parameters with shape (M). 'M' is the number of basis functions.
    points : 1D np.ndarray
        Radial coordinates of grid points, with shape (N,).
    weights : 1D np.ndarray
        Weights for integration, including the angular part (4πr²), with shape (N,).
    threshold : float
        Convergence threshold for the iterative process.
    density_cutoff : float
        Density values below this cutoff are considered invalid.
    negative_cutoff : float
        The value less than `negative_cutoff` is treated as negative.
    population_cutoff : float
        The criteria for the difference between the sum of propars and reference population.


    Returns
    -------
    1D np.ndarray
        Optimized proatom parameters.

    Raises
    ------
    RuntimeError
        If the inner iteration does not converge.

    """
    pro_shells = bs_funcs * propars[:, None]
    pro_density = np.einsum("ij->j", pro_shells)
    sick = (rho < density_cutoff) | (pro_density < density_cutoff)
    ratio = np.divide(rho, pro_density, out=np.zeros_like(rho), where=~sick)
    propars[:] = np.einsum("p,ip->i", weights, pro_shells * ratio)
    return propars

In [ ]:
def customized_self_consistent_method():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = customize_self_consistent_solver
    part = LinearISAWPart(**kwargs)
    part.do_all()
    print_results(part)


customized_self_consistent_method()

One can also apply customized basis functions using the `ExpBasisFuncHelper` class. First, we need to define the basis functions, which should adhere to the following general format:

$$
  f_{ak} = c_{ak} \exp^{-\alpha_{ak} |r|^{n_{ak}}}
$$

Here, $c_{ak}$ represents the prefactors and the pro-atom parameters to be determined; $\alpha_{ak}$ is the exponential coefficient; and $n_{ak}$ corresponds to the power of the exponential function. The subscripts $a$ and $k$ denote the indices of atoms and basis functions, respectively.

We initialize `ExpBasisFuncHelper` using the default method, and therefore need to provide the $\alpha_{ak}$ (`exponents`), $n_{ak}$ (`exponents_orders`), and initial values of $c_{ak}$ (`initials`). To illustrate the functionality, we use three exponential functions with different $n$ values, i.e., 1 (Slater type), 1.5, and 2 (Gaussian type).

In [ ]:
bs_dict = {
    "exponents_orders": {1: [1, 1.5, 2], 8: [1, 1.5, 2]},
    "exponents": {1: [0.1, 0.2, 0.3], 8: [1.0, 2.0, 3.0]},
    "initials": {1: [0.33] * 3, 8: [0.33] * 3},
}
basis_helper = ExpBasisFuncHelper(**bs_dict)

It should be noted that one can also initialize a `ExpBasisFuncHelper` through a `json` file by calling `ExpBasisFuncHelper.from_json('json_file_path.json')`.

Next, we can apply the newly customized basis functions and the self-consistent solver. It is important to note

In [ ]:
def customized_basis_functions():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = customize_self_consistent_solver
    kwargs["basis_func"] = basis_helper
    part = LinearISAWPart(**kwargs)
    part.do_all()
    print_results(part)


customized_basis_functions()